# Day 2
## Part 1 Problem Statement
<p>You're launched high into the atmosphere! The apex of your trajectory just barely reaches the surface of a large island floating in the sky. You gently land in a fluffy pile of leaves. It's quite cold, but you don't see much snow. An Elf runs over to greet you.</p>
<p>The Elf explains that you've arrived at <em>Snow Island</em> and apologizes for the lack of snow. He'll be happy to explain the situation, but it's a bit of a walk, so you have some time. They don't get many visitors up here; <span title="No, the Elf's name is not 'WOPR'. It's Joshua.">would you like to play a game</span> in the meantime?</p>
<p>As you walk, the Elf shows you a small bag and some cubes which are either red, green, or blue. Each time you play this game, he will hide a secret number of cubes of each color in the bag, and your goal is to figure out information about the number of cubes.</p>
<p>To get information, once a bag has been loaded with cubes, the Elf will reach into the bag, grab a handful of random cubes, show them to you, and then put them back in the bag. He'll do this a few times per game.</p>
<p>You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the <code>11</code> in <code>Game 11: ...</code>) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like <code>3 red, 5 green, 4 blue</code>).</p>
<p>For example, the record of a few games might look like this:</p>
<pre><code>Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
</code></pre>
<p>In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.</p>
<p>The Elf would first like to know which games would have been possible if the bag contained <em>only 12 red cubes, 13 green cubes, and 14 blue cubes</em>?</p>
<p>In the example above, games 1, 2, and 5 would have been <em>possible</em> if the bag had been loaded with that configuration. However, game 3 would have been <em>impossible</em> because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been <em>impossible</em> because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get <code><em>8</em></code>.</p>
<p>Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. <em>What is the sum of the IDs of those games?</em></p>

## Part 1 Strategy

Seems easy. Just check each line for invalid pulls, invalid being if any of the cubes pulled exceed what is in the bag.

Most of this problem is parsing the data. I need to convert the input data into something like this:

```
{1: [
        {
        blue: 3,
        red: 4,
        green: 0,
        },
        {
        blue: 6,
        red: 1,
        green: 2,
        },
    ],
2: ...
}
```
then just check each numbered key for any invalid games.

In [3]:
test_part1 = """Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"""

In [28]:
import re # use regex to find some information in strings

def get_round_dict_from_str(round: str) -> dict[str,int]:
    round_dict = {"red": 0, "blue": 0, "green": 0}
    round_data = round.split(",")
    for pull in round_data:
        pull = pull.strip()
        number, color = re.findall("([0-9]+) (blue|red|green)", pull)[0] # returns a tuple, number then color
        number = int(number)
        round_dict[color] = number
    return round_dict

def get_single_game_stats_from_string(game: str) -> list[dict[str,int]]:
    game = game.split(":")[1].strip() # get rid of game part, strip whitespace from sides
    rounds_list = game.split(";")
    rounds_results = []
    for round in rounds_list:
        round_result = get_round_dict_from_str(round)
        rounds_results.append(round_result)
    return rounds_results
    
def get_games_stats_from_string(puzzle_string: str) -> dict[int, list[dict[str,int]]]:
    games_by_line = puzzle_string.split("\n")
    games_stats = {}
    for game in games_by_line:
        game_num = int(re.findall("Game ([0-9]+):", game)[0]) # returns only the number
        list_of_games = get_single_game_stats_from_string(game)
        games_stats[game_num] = list_of_games
    return games_stats
        
def part1(puzzle_string: str) -> int:
    limit_lookup = {
        "red":12,
        "blue":14,
        "green":13,
    }
    games_stats_lookup = get_games_stats_from_string(puzzle_string)
    valid_games = []
    for game_num, stats_games in games_stats_lookup.items():
        game_valid = True
        for round in stats_games:
            for color in ["red","blue","green"]:
                if round[color] > limit_lookup[color]:
                    game_valid = False
        if game_valid:
            valid_games.append(game_num)
    return sum(valid_games)
    

In [29]:
part1(test_part1) # should be 8

8

In [31]:
with open("day2input.txt","r") as f:
    part1_data_real = f.read()

part1(part1_data_real) # got em!

2449

<h2 id="part2">--- Part Two ---</h2><p>The Elf says they've stopped producing snow because they aren't getting any <em>water</em>! He isn't sure why the water stopped; however, he can show you how to get to the water source to check it out for yourself. It's just up ahead!</p>
<p>As you continue your walk, the Elf poses a second question: in each game you played, what is the <em>fewest number of cubes of each color</em> that could have been in the bag to make the game possible?</p>
<p>Again consider the example games from earlier:</p>
<pre><code>Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
</code></pre>
<ul>
<li>In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.</li>
<li>Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.<li>
<li>Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.</li>
<li>Game 4 required at least 14 red, 3 green, and 15 blue cubes.</li>
<li>Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.</li>
</ul>
<p>The <em>power</em> of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is <code>48</code>. In games 2-5 it was <code>12</code>, <code>1560</code>, <code>630</code>, and <code>36</code>, respectively. Adding up these five powers produces the sum <code><em>2286</em></code>.</p>
<p>For each game, find the minimum set of cubes that must have been present. <em>What is the sum of the power of these sets?</em></p>

## Part 2 strats
This seems easy, and I can reuse my data structure from before. For each game, get the maximum number for each color that was pulled, then multiply them. Sum this up.

In [32]:
def part2(puzzle_string: str) -> int:
    games_stats_lookup = get_games_stats_from_string(puzzle_string)
    powers_by_game = []
    for game_num, stats_games in games_stats_lookup.items():
        red = max([pull["red"] for pull in stats_games])
        green = max([pull["green"] for pull in stats_games])
        blue = max([pull["blue"] for pull in stats_games])
        powers_by_game = red*blue*green
    return sum(powers_by_game)

In [ ]:
part2(te